In [1]:
#TODO
#Add teams and then this is pretty much done subject to cleaning up

import sklearn
from sklearn.model_selection import train_test_split
import sqlite3
from sqlite3 import Error
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from pulp import *
import re
import scipy
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn import svm
from mealpy.swarm_based.MFO import BaseMFO
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [132]:
REQUIRED_FIELDS = ['assists','bonus','bps','clean_sheets','element_type','goals_conceded','goals_scored','minutes','points_per_game',
'selected_by_percent','total_points_old','yellow_cards',
#                    'penalties_saved','saves','dreamteam_count','red_cards'
                   'total_points_new']
REQUIRED_FIELDS_TP = ['assists','bonus','bps','clean_sheets','element_type','goals_conceded','goals_scored','minutes','points_per_game',
'selected_by_percent','total_points','yellow_cards'
#                       ,'penalties_saved','saves','dreamteam_count','red_cards'
                     ]
REQUIRED_FIELDS_GW = ['assists','bonus','bps','clean_sheets','goals_conceded','goals_scored','minutes',
'yellow_cards',
#                       'saves','red_cards',
                      'total_points','name']
REQUIRED_FIELDS_GW_NN = [
#     'assists','bonus','bps','clean_sheets','goals_conceded','goals_scored',
    'minutes',
# 'yellow_cards',
#                          'saves','red_cards',
                         'total_points']
REQUIRED_FIELDS_GW34 = [
#     'assists34','bonus34','bps34','clean_sheets34','goals_conceded34',
#                         'goals_scored34',
                        'minutes34',
# 'yellow_cards34',
#                         'saves34','red_cards34',
                        'total_points']
REQUIRED_FIELDS_GW35 = [s+"35"for s in REQUIRED_FIELDS_GW_NN]
REQUIRED_FIELDS_GW36 = [s+"36"for s in REQUIRED_FIELDS_GW_NN]
REQUIRED_FIELDS_GW37 = [s+"37"for s in REQUIRED_FIELDS_GW_NN]
REQUIRED_FIELDS_GW38 = [s+"38"for s in REQUIRED_FIELDS_GW_NN]

season16 = pd.read_csv("data/2016-17/players_raw.csv")
season17 = pd.read_csv("data/2017-18/players_raw.csv")
season18 = pd.read_csv("data/2018-19/players_raw.csv")
season19 = pd.read_csv("data/2019-20/players_raw.csv")
season20 = pd.read_csv("data/2020-21/players_raw.csv")

season16GWs = pd.read_csv("data/2016-17/gws/gw34.csv", engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW)
season17GWs = pd.read_csv("data/2017-18/gws/gw34.csv", engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW)
season18GWs = pd.read_csv("data/2018-19/gws/gw34.csv", engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW)
season19GWs = pd.read_csv("data/2019-20/gws/gw42.csv", engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW)
for i in range(35,39):
    season16GWs = pd.merge(season16GWs, pd.read_csv("data/2016-17/gws/gw"+str(i)+".csv",engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW),on="name",suffixes=["",i], how = 'inner')
    season17GWs = pd.merge(season17GWs, pd.read_csv("data/2017-18/gws/gw"+str(i)+".csv",engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW),on="name",suffixes=["",i], how = 'inner')
    season18GWs = pd.merge(season18GWs, pd.read_csv("data/2018-19/gws/gw"+str(i)+".csv",engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW),on="name",suffixes=["",i], how = 'inner')
    season19GWs = pd.merge(season19GWs, pd.read_csv("data/2019-20/gws/gw"+str(i+8)+".csv",engine="python",
                         usecols = lambda x:x.lower() in REQUIRED_FIELDS_GW),on="name",suffixes=["",i], how = 'inner')

season19Short = season19[['first_name','second_name','total_points']]
season18Short = season18[['first_name','second_name','total_points']]
season17Short = season17[['first_name','second_name','total_points']]

season1617 = pd.merge(season16, season17Short,on=['first_name','second_name'],suffixes = ['_old','_new'], how = 'inner' )
season1718 = pd.merge(season17, season18Short,on=['first_name','second_name'],suffixes = ['_old','_new'], how = 'inner' )
season1819 = pd.merge(season18, season19Short,on=['first_name','second_name'],suffixes = ['_old','_new'], how = 'inner' )
# print(season16GWs["name"])
splitName16 = season16GWs["name"].str.split("_")
splitName17 = season17GWs["name"].str.split("_")
splitName18 = season18GWs["name"].str.split("_")
splitName19 = season19GWs["name"].str.split("_")
# print(splitName16)
season1617["nameFixed"] = season1617["first_name"]+" "+season1617["second_name"]
season1718["nameFixed"] = season1718["first_name"]+" "+season1718["second_name"] 
season1819["nameFixed"] = season1819["first_name"]+" "+season1819["second_name"] 
season19["nameFixed"] = season19["first_name"]+" "+season19["second_name"]
season20["nameFixed"] = season20["first_name"]+" "+season20["second_name"]
def split(toSplit,season, num):
#     print(toSplit)
    for idx,name in enumerate(toSplit):
        if(num):
            name = name[:-1]
#         print(name)
#         sys.exit()
        part = ""
        for parts in name:
            part += parts
            part += " "
        toSplit[idx] =  part.rstrip()
    season["nameFixed"] = toSplit
    return season
season16GWs = split(splitName16, season16GWs,0)
season17GWs = split(splitName17, season17GWs,0)
season18GWs = split(splitName18, season18GWs,1)
season19GWs = split(splitName19, season19GWs,1)
# print(season16GWs["nameFixed"])
# print(season17GWs["nameFixed"])
# print(season18GWs["nameFixed"])
# print(season17GWs["nameFixed"])
# print(season1718["nameFixed"])
season1617 = pd.merge(season1617, season16GWs, on="nameFixed",suffixes=["","34"], how = 'inner')
season1718 = pd.merge(season1718, season17GWs, on="nameFixed",suffixes=["","34"], how = 'inner')
season1819 = pd.merge(season1819, season18GWs, on="nameFixed",suffixes=["","34"], how = 'inner')
season1920 = pd.merge(season19, season19GWs, on="nameFixed",suffixes=["","34"], how = 'inner')
# print(season19GWs)
# print(season1819)
seasonAll = season1617.append(season1718)
seasonAll = seasonAll.append(season1819)
# for column in seasonAll.columns:
#     print(column)
# print(seasonAll.columns)

# print(seasonAll)
#TODO Update fields
fields = REQUIRED_FIELDS+REQUIRED_FIELDS_GW34+REQUIRED_FIELDS_GW35+REQUIRED_FIELDS_GW36+REQUIRED_FIELDS_GW37+REQUIRED_FIELDS_GW38
seasonAllRequiredFields = seasonAll[fields]
# print(seasonAllRequiredFields)
fields2 = REQUIRED_FIELDS_TP+REQUIRED_FIELDS_GW34+REQUIRED_FIELDS_GW35+REQUIRED_FIELDS_GW36+REQUIRED_FIELDS_GW37+REQUIRED_FIELDS_GW38
lastSeason = season1920[fields2]

print(lastSeason)
seasonAllRequiredFields['goals_scored'] = (seasonAllRequiredFields['goals_scored']/seasonAllRequiredFields['minutes']).fillna(0)
seasonAllRequiredFields['goals_conceded'] = (seasonAllRequiredFields['goals_conceded']/seasonAllRequiredFields['minutes']).fillna(1)
seasonAllRequiredFields['goals_scored'] .replace([np.inf, -np.inf],0)
seasonAllRequiredFields['goals_conceded'] .replace([np.inf, -np.inf],70)

     assists  bonus  bps  clean_sheets  element_type  goals_conceded  \
0          2      2  242             4             2              17   
1          0      4  204             4             2              18   
2          2      1  325             4             2              25   
3          2      3  229             3             2              20   
4          0      5  305             4             2              25   
..       ...    ...  ...           ...           ...             ...   
647        0      0    0             0             2               0   
648        0      0    0             0             2               0   
649        0      0    0             0             3               0   
650        1      2   76             3             3               0   
651        0      0    0             0             4               0   

     goals_scored  minutes  points_per_game  selected_by_percent  ...  \
0               0     1205              2.9                  0

/home/jordan/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
/home/jordan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jordan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

0      0.027972
1      0.027972
2      0.027972
3      0.027972
4      0.011947
         ...   
559    0.014334
560    1.000000
561    1.000000
562    0.000000
563    0.000000
Name: goals_conceded, Length: 1598, dtype: float64

In [133]:
# seasonAllRequiredFields.drop(['element_type'], axis='columns')
x = seasonAllRequiredFields.iloc[:,seasonAllRequiredFields.columns != 'total_points_new']
y = seasonAllRequiredFields.iloc[:,seasonAllRequiredFields.columns == 'total_points_new']
print(x)
print(y)
y=y["total_points_new"].ravel()
X_train, X_test, y_train, y_test = train_test_split(x, y, 
test_size=0.8, random_state=12421)

     assists  bonus  bps  clean_sheets  element_type  goals_conceded  \
0          0      0   18             0             1        0.027972   
1          0      0   18             0             1        0.027972   
2          0      0   18             0             1        0.027972   
3          0      0   18             0             1        0.027972   
4          0      2  660            12             1        0.011947   
..       ...    ...  ...           ...           ...             ...   
559        0      1  209             3             3        0.014334   
560        0      0    0             0             2        1.000000   
561        0      0    0             0             2        1.000000   
562        0      0    3             0             2        0.000000   
563        0      0    3             0             2        0.000000   

     goals_scored  minutes  points_per_game  selected_by_percent  ...  \
0        0.000000      143              1.0                  0

In [156]:
md = RandomForestRegressor(bootstrap=True, criterion="mse", max_depth=500, n_estimators=10000,
                          max_features ="log2", min_samples_leaf=10, min_samples_split=9 )
md.fit(x, y)
prediction = cross_val_score(md, x, y, cv=10, scoring="r2")
print(prediction)
# print(classification_report(y_test,prediction))
# print(accuracy_score(y_test, prediction))
#~40r2

[0.27649879 0.31263189 0.17151505 0.55434076 0.4708692  0.40069257
 0.31662506 0.30454501 0.41261436 0.47028999]


In [158]:
# pos = [1.2424409, 0.88343281, 0.77583797, 1.30566707, 0.52402922, 1.4493438,
#  1.30963066, 0.61526618, 0.79446204, 1.15044055, 0.95248908, 1.35647921,
#  0.64200355, 0.92972736, 1.30896911, 0.94015849]
# mask = [1,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0]
# mask = np.reshape(pos,(1,-1))
# mask = np.round(mask)
# mask = mask[0]
xConv = x.to_numpy()
# xFeatures = xConv[:,mask==1]
xFeatures = xConv
space = {'criterion' : hp.choice("criterion",['mse','mae']),
         'max_depth' : hp.choice("depth",[10,25,50,100,250,500,1000]),
         'min_samples_split' : hp.choice("minSamplesSplit",[2,3,4,5,6,7,8,9,10,11,12]),
         'min_samples_leaf' : hp.choice("minSamplesLeaf",[1,2,3,4,5,6,7,8,9,10]),
         'max_features' : hp.choice("maxFeat",["auto","log2",None]),
         'bootstrap':hp.choice("bootstrap",[True,False]),
         'n_estimators':hp.choice("estimators",[100,250,500])}

def objective(space):
    md = RandomForestRegressor(criterion=space['criterion'],
                                max_depth=space['max_depth'],
                                min_samples_split=space["min_samples_split"],
                                min_samples_leaf=space["min_samples_leaf"],
                                max_features=space["max_features"],
                                bootstrap=space["bootstrap"],
                                n_estimators=space["n_estimators"],
                                n_jobs=-1)
    accuracy = cross_val_score(md,xFeatures,y, cv=10, scoring="r2", n_jobs=-1).mean()
    return{'loss': -accuracy,'status':STATUS_OK}

best = fmin(fn=objective,
            space=space,
            algo= tpe.suggest,
            max_evals = 100,
            trials = Trials())

best

100%|██████████| 100/100 [39:20<00:00, 23.61s/trial, best loss: -0.37419698295535475] 


{'bootstrap': 1,
 'criterion': 0,
 'depth': 0,
 'estimators': 2,
 'maxFeat': 1,
 'minSamplesLeaf': 0,
 'minSamplesSplit': 1}

In [159]:
def MFOObjectiveFuncRF(xPassed, alpha=0.0):
    colLen = len(x.columns)
    mask = np.reshape(xPassed,(1,-1))
    mask = np.round(mask)
    mask = mask[0]
    xConv = x.to_numpy()
#     print(xConv)
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
#     print(x_test)
    md = RandomForestRegressor(criterion="mse",n_estimators = 50,\
                            max_depth=10, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=3, n_jobs = -1)
    md.fit(x_test,y)
    performance = cross_val_score(md, x, y, cv=10, scoring="r2", n_jobs=-1).mean()
    result = (1- performance)
    print(result)
    return result


In [ ]:
print(sorted(sklearn.metrics.SCORERS.keys()))
md =BaseMFO(MFOObjectiveFuncRF,71,(0,1),50, 50)
pos, fit, loss = md._train__()
print(fit)          
# performance = md.predict(x_test)
# performance = accuracy_score(y, performance)
print(pos)    
mask = np.reshape(pos/2,(1,-1))
mask = np.round(mask)
mask = mask[0]
print(mask)

In [134]:
# pos = np.array([ 0.7389128,   0.76441606,  0.85684991,  0.49031168,  0.40555134,  0.48250744,
#   0.22690716,  0.32744212,  0.3572608,   0.09615164,  1.41844831,  0.92160598,
#  -1.33506296,  1.01041817, -0.38325607,  3.46366103,  1.73379808,  0.71373228,
#   1.03421529,  0.71330065, -0.36651596 , 1.68473145,  1.2973866 ,  0.7752127,
#   0.16319451,  0.71587942,  0.52931191 , 1.93985163,  0.79792679,  0.92716469,
#   0.84387973,  0.5697812 , -0.37895981,  0.95754319,  0.03811236,  0.44721581,
#   0.75565891,  0.3755427 ,  0.53478808,  2.86278399,  0.07467774,  0.66008522,
#   0.74688257,  0.53851311,  2.61820797,  1.04552684,  0.45566751,  0.24148105,
#   1.12184518,  0.74540933,  0.43062156, -0.20530645,  1.93784711,  0.35645021,
#  -0.37406225,  0.18110619,  1.66755575,  0.3562889,   1.12560845,  1.3669794,
#   0.60372351,  0.59697854,  0.49535092,  0.18373853,  0.53701486,  0.9145044,
#   0.61721493,  0.74700258,  0.69320287,  0.27082067 , 0.29972997])
# # pos = [1.2424409, -0.88343281, -0.77583797, 1.30566707, -0.52402922, 1.4493438,
# #  1.30963066, -0.61526618, -0.79446204, 1.15044055, 0.95248908, 1.35647921,
# #  -0.64200355, 0.92972736, 1.30896911, 0.94015849]
# # mask = [1,0,0,1,0,1,1,0,0,1,1,1,0,0,1,0]

# mask = np.reshape(pos/2,(1,-1))
# mask = np.round(mask)
# mask = mask[0]
# print(mask)
# print(x)
# xConv = x.to_numpy()

# xFeatures = xConv[:,mask>=0]
md = RandomForestRegressor(bootstrap=True, criterion="mse", max_depth=15, n_estimators=10000,
                          max_features =None, min_samples_leaf=2, min_samples_split=3 )
md.fit(x,y)
# xConv = lastSeason.to_numpy()
# xFeatures = xConv[:,mask>=0]
prediction = md.predict(lastSeason)
print(prediction.round())
print(len(prediction))
print(len(xFeatures))
season1920['prediction'] = prediction
# season20 = pd.merge(season20, season1920, on=['first_name','second_name'],suffixes=["","IGNORE"], how = 'inner')
season1920.to_csv('predictions.csv')
# season19['prediction'] = prediction
# season19.to_csv('predictions.csv')

[ 63.  67.  59.  47.  63.  11.  10.  14.  14.  32. 157.  87.  57.  60.
  79.  25.  77.  64.  24.  49.  93.  73.  67.  59.  72.  78.  59.  60.
  60.  60.  74.  19.  12.  16.  14.   7.  14.  70.  14.  61.  64.  18.
  29.  94. 101.  68.  93.   9.   9.  24.  24.  28.  53.  22.  76.  77.
  25.  97.  75.  54.  48.  89.  93.   8.  62.  11.  81.  48.  11.  67.
  72.  13.  53.  54.  53.  71.  97.  24.  76.  67.  72.  33.  35.  16.
   9.  67.  44.  69.   6.  62.  16.  64.  69.  61.  67.  19. 100.  78.
   9.   9.   9.  55.  54.  55.  14.   6. 104.  14.  64.  24.  47.  99.
  36.  84.  53.   9.   9.  52.  79.  47.   8.  64.  78.  76.  85.  97.
  87.  66.  17.  55.  22.  72.  31.  83. 102.  59.  76.  65.  75.  13.
  83.  95.  54.  85.  37.  55.  94.  58.   9.  79.  89.  59.  26.  79.
  87.   8.   9.  10.  14.  57.  14.  14.  13.   7.   9.  88.  82.  91.
  57.  57.  17.  76.  81.  19. 115.  77.  33.  55.  84.  90.  56.   9.
 122.  63.  66. 132.  64.   9. 119.  61.  60.  31.  10.  15.  11.  75.
  58. 

In [18]:
import pickle
with open("model.pkl", 'wb') as f:
    pickle.dump(md, f)

In [75]:
# xConv = lastSeason.to_numpy()
# xFeatures = xConv[:,mask>=0]
prediction = md.predict(xFeatures)
print(prediction.round())
print(len(prediction))
print(len(season20))
season1920['prediction'] = prediction
# season20 = pd.merge(season20, season1920, on=['first_name','second_name'],suffixes=["","IGNORE"], how = 'inner')
season1920.to_csv('predictions.csv')

ValueError: Number of features of the model must match the input. Model n_features is 71 and input n_features is 70 

In [2]:
import numpy as np
import pandas as pd
import pulp
import os
import argparse
import warnings


#needs use_now_cost
#start_cost
#position
#team_id
#full_name
#prediction
def add_position_dummy(df):
    for p in df.position.unique():
        df['is_' + str(p).lower()] = np.where(df.position == p, int(1), int(0))
    return df


def add_team_dummy(df):
    for t in df.teamold.unique():
        df['team_' + str(t).lower()] = np.where(df.teamold == t, int(1), int(0))
    return df


def get_optimal_squad(data, formation='2-5-5-3', budget=100.0, season='2019/20',
                      use_now_cost=True, optimise_on='prediction'):
    min_player_cost = 4.
    n_players = sum(int(i) for i in formation.split('-'))
    max_budget = 100 - (15 - n_players)*min_player_cost

    if budget is None:
        budget = max_budget

    if budget > max_budget:
        warnings.warn('Supplied budget exceeds expected maximum of '
                      '{0}'.format(max_budget))

    season_stats = (
        data
        .reset_index()
        .assign(cost=lambda df: (df.now_costold / 10.) if use_now_cost else (df.start_cost / 10.))
        .pipe(add_position_dummy)
        .pipe(add_team_dummy)
    )

    players = season_stats.nameFixed

    # initalise the problem
    fpl_problem = pulp.LpProblem('FPL', pulp.LpMaximize)

    # create a dictionary of pulp variables with keys from names
    x = pulp.LpVariable.dict('x_ % s', players, lowBound=0, upBound=1,
                             cat=pulp.LpInteger)

    # player score data
    player_points = dict(
        zip(season_stats.nameFixed, np.array(season_stats[optimise_on])))

    # objective function
    fpl_problem += sum([player_points[i] * x[i] for i in players])

    # constraints
    position_names = ['gk', 'def', 'mid', 'fwd']
    position_constraints = [int(i) for i in formation.split('-')]

    constraints = dict(zip(position_names, position_constraints))
    constraints['total_cost'] = budget
    constraints['team'] = 3

    # could get straight from dataframe...
    player_cost = dict(zip(season_stats.nameFixed, season_stats.cost))
    player_position = dict(zip(season_stats.nameFixed, season_stats.position))
    player_gk = dict(zip(season_stats.nameFixed, season_stats.is_goalkeeper))
    player_def = dict(zip(season_stats.nameFixed, season_stats.is_defender))
    player_mid = dict(zip(season_stats.nameFixed, season_stats.is_midfielder))
    player_fwd = dict(zip(season_stats.nameFixed, season_stats.is_forward))

    # apply the constraints
    fpl_problem += sum([player_cost[i] * x[i] for i in players]) <= float(constraints['total_cost'])
    fpl_problem += sum([player_gk[i] * x[i] for i in players]) == constraints['gk']
    fpl_problem += sum([player_def[i] * x[i] for i in players]) == constraints['def']
    fpl_problem += sum([player_mid[i] * x[i] for i in players]) == constraints['mid']
    fpl_problem += sum([player_fwd[i] * x[i] for i in players]) == constraints['fwd']

    for t in season_stats.teamold:
        player_team = dict(
            zip(season_stats.nameFixed, season_stats['team_' + str(t)]))
        fpl_problem += sum([player_team[i] * x[i] for i in players]) <= constraints['team']

    # solve the thing
    fpl_problem.solve()

    total_points = 0.
    total_cost = 0.
    optimal_squad = []
    for p in players:
        if x[p].value() != 0:
            total_points += player_points[p]
            total_cost += player_cost[p]

            optimal_squad.append({
                'name': p,
                'position': player_position[p],
                'cost': player_cost[p],
                'points': player_points[p]
            })

    solution_info = {
        'formation': formation,
        'total_points': total_points,
        'total_cost': total_cost
    }
    optimal_squad = pd.DataFrame(optimal_squad)

    return optimal_squad, solution_info


In [4]:
# df2 = season1920
df2 = pd.read_csv("gw_prediction.csv")
print(df2)
positions_csv = pd.read_csv("data/2020-21/players_raw.csv", engine='python')
positions_csv = positions_csv[["first_name","second_name","chance_of_playing_next_round",
                              "element_type","now_cost","team"]]
positions_csv["nameFixed"] = positions_csv["first_name"]+" "+positions_csv["second_name"]
df2["nameFixed"] = df2["first_name"]+" "+df2["second_name"]

# for idx,name in enumerate(stuffs):
#     name = name[:-1]
#     part = ""
#     for parts in name:
#         part += parts
#         part += " "
#     stuffs[idx] =  part.rstrip()
# #add stuffs, chance of playing, positions

# df2["nameFixed"] = stuffs
# print(df2["nameFixed"])
# print(positions_csv["nameFixed"])
df2 = pd.merge(df2,positions_csv,on="nameFixed",suffixes=["new","old"],how="inner")
print(df2.columns)
df2["position"] = df2["element_typeold"]
df2["is_goalkeeper"] = np.where(df2["position"]==1,1,0)
df2["is_defender"] = np.where(df2["position"]==2,1,0)
df2["is_midfielder"] = np.where(df2["position"]==3,1,0)
df2["is_forward"] = np.where(df2["position"]==4,1,0)
# results = df2["chance_of_playing_next_round"]=="100"
# df2 = df2[results]
# print(results)
squad, soln = get_optimal_squad(df2,formation='1-3-5-2', budget=83.5)
print(squad)
squad.columns = [col.capitalize() for col in squad.columns]

# Display output
print(squad.to_string(index=False))
print('')  # extra line break
for key, val in soln.items():
    pretty_key = key.replace('_', ' ').capitalize()
    print('{0}:\t{1}'.format(pretty_key, val))

     Unnamed: 0                       name  assists  bonus  bps  clean_sheets  \
0             0           Dennis_Praet_225        0      1   29             0   
1             1         Oliver_Norwood_349        0      0    0             0   
2             2       Giovani_Lo Celso_403        0      0    2             0   
3             3  Alisson_Ramses Becker_252        0      2   41             1   
4             4           Paul_Dummett_331        0      0    0             0   
..          ...                        ...      ...    ...  ...           ...   
534         534      Marcus_Bettinelli_183        0      0    0             0   
535         535          Michael_Keane_156        0      0   28             0   
536         536      Benjamin_Chilwell_232        0      0    0             0   
537         537            Jordan_Ayew_140        1      0   13             0   
538         538       Stuart_Armstrong_368        0      0   11             0   

     creativity  element  f

KeyError: 'first_name'

In [37]:
optimalTeam(season20)

     assists  bonus  bps chance_of_playing_next_round  \
2          2      1  222                          100   
3          2      3  216                          100   
4          0      5  305                          100   
10         3     29  601                          100   
11         4      9  277                          100   
..       ...    ...  ...                          ...   
607        0      8  516                         None   
610        1      4  372                          100   
611        7     11  508                         None   
613        7     14  460                          100   
616        0      6  331                          100   

    chance_of_playing_this_round  clean_sheets    code  cost_change_event  \
2                            100             2  111457                  0   
3                            100             3  154043                  0   
4                            100             4   39476                  0   
10     

/home/jordan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
